In [ ]:
# Install all required dependencies
!pip install pandas numpy scikit-learn gradio matplotlib seaborn gTTS folium reportlab pyngrok requests openpyxl

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import gradio as gr
import matplotlib
matplotlib.use('agg')  # Use non-interactive backend for Colab
import matplotlib.pyplot as plt
import seaborn as sns
from gtts import gTTS
import os
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import folium
from google.colab import files
import logging
from pyngrok import ngrok
import socket
import psutil
import requests
from IPython.display import Image, display

# Set up logging for debugging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Ensure output directory exists
output_dir = "/content/crop_recommender"
os.makedirs(output_dir, exist_ok=True)

# Helper function to find an available port
def find_available_port(start_port, end_port):
    for port in range(start_port, end_port + 1):
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            try:
                s.bind(("localhost", port))
                return port
            except OSError:
                continue
    return None

# Helper function to kill processes on a port
def kill_process_on_port(port):
    try:
        for conn in psutil.net_connections():
            if conn.laddr.port == port:
                process = psutil.Process(conn.pid)
                process.terminate()
                logger.info(f"Killed process {conn.pid} on port {port}")
    except Exception as e:
        logger.error(f"Error killing process on port {port}: {e}")

# Helper function to terminate existing ngrok sessions
def terminate_ngrok_sessions(api_key=None):
    if not api_key:
        logger.warning("No ngrok API key provided. Manually terminate sessions at https://dashboard.ngrok.com/agents")
        return
    try:
        headers = {"Authorization": f"Bearer {api_key}", "Ngrok-Version": "2"}
        response = requests.get("https://api.ngrok.com/sessions", headers=headers)
        if response.status_code == 200:
            sessions = response.json().get("sessions", [])
            for session in sessions:
                session_id = session["id"]
                requests.delete(f"https://api.ngrok.com/sessions/{session_id}", headers=headers)
                logger.info(f"Terminated ngrok session: {session_id}")
        else:
            logger.error(f"Failed to fetch ngrok sessions: {response.text}")
    except Exception as e:
        logger.error(f"Error terminating ngrok sessions: {e}")

# Helper function to list generated files
def list_output_files():
    try:
        files = os.listdir(output_dir)
        return f"Generated files in {output_dir}: {', '.join(files) if files else 'None'}"
    except Exception as e:
        logger.error(f"Error listing files: {e}")
        return f"Error listing files: {e}"

# Helper function to parse yield strings
def parse_yield(yield_str):
    try:
        parts = yield_str.split('-')
        low = float(parts[0].replace('t/ha', '').strip())
        high = float(parts[1].replace('t/ha', '').strip())
        return low, high
    except Exception as e:
        logger.error(f"Error parsing yield '{yield_str}': {e}")
        return 1.0, 2.0  # Fallback values

# Helper function to parse water strings
def parse_water(water_str):
    try:
        if '-' in water_str:
            parts = water_str.split('-')
            high = float(parts[1].replace('mm', '').strip())
        else:
            high = float(water_str.replace('mm', '').strip())
        return high
    except Exception as e:
        logger.error(f"Error parsing water '{water_str}': {e}")
        return 300.0  # Fallback value

# Dataset with 30 Uttar Pradesh cities
data = {
    "city": ["Lucknow", "Kanpur", "Varanasi", "Agra", "Meerut", "Prayagraj", "Ghaziabad", "Jhansi", "Aligarh", "Bareilly",
             "Gorakhpur", "Noida", "Firozabad", "Mathura", "Moradabad", "Saharanpur", "Faizabad", "Muzaffarnagar", "Rampur", "Azamgarh",
             "Basti", "Deoria", "Sitapur", "Lakhimpur", "Hathras", "Mirzapur", "Bulandshahr", "Ayodhya", "Gonda", "Barabanki"],
    "temperature": [25, 30, 22, 35, 28, 20, 18, 26, 32, 29, 24, 27, 33, 21, 19, 23, 31, 20, 28, 26, 25, 24, 27, 26, 28, 22, 29, 24, 25, 27],
    "humidity": [80, 60, 90, 50, 65, 85, 70, 75, 55, 60, 72, 78, 66, 88, 82, 70, 62, 85, 58, 64, 76, 79, 68, 71, 67, 81, 63, 77, 74, 69],
    "rainfall": [200, 150, 300, 100, 120, 250, 280, 190, 130, 160, 210, 170, 220, 260, 240, 180, 140, 270, 110, 155, 195, 205, 175, 185, 165, 230, 145, 200, 190, 180],
    "soil": ['clay', 'sandy', 'loamy', 'black', 'red', 'clay', 'sandy', 'loamy', 'black', 'red', 'clay', 'sandy', 'loamy', 'black', 'red',
             'clay', 'sandy', 'loamy', 'black', 'red', 'clay', 'sandy', 'loamy', 'black', 'red', 'clay', 'sandy', 'loamy', 'black', 'red'],
    "season": ['kharif', 'rabi', 'summer', 'kharif', 'rabi', 'summer', 'kharif', 'rabi', 'summer', 'kharif', 'rabi', 'summer', 'kharif', 'rabi', 'summer',
               'kharif', 'rabi', 'summer', 'kharif', 'rabi', 'summer', 'kharif', 'rabi', 'summer', 'kharif', 'rabi', 'summer', 'kharif', 'rabi', 'summer'],
    "crop": ['Rice', 'Wheat', 'Tulsi', 'Mango', 'Rose', 'Tomato', 'Pea', 'Marigold', 'Banana', 'Mint', 'Apple', 'Onion', 'Neem', 'Lotus', 'Guava',
             'Maize', 'Gram', 'Mustard', 'Sugarcane', 'Cucumber', 'Jowar', 'Bajra', 'Ragi', 'Moong', 'Arhar', 'Potato', 'Cauliflower', 'Jasmine', 'Litchi', 'Peepal']
}
df = pd.DataFrame(data)

# Encode categorical variables
df_encoded = pd.get_dummies(df, columns=['city', 'soil', 'season'])
X = df_encoded.drop("crop", axis=1)
y = df["crop"]
model = RandomForestClassifier(random_state=42)
model.fit(X, y)
training_columns = X.columns.tolist()

# Crop rotation suggestions for soil health
crop_rotation = {
    "Rice": ["Wheat", "Gram", "Mustard"],
    "Wheat": ["Rice", "Moong", "Sugarcane"],
    "Tulsi": ["Marigold", "Mint", "Jasmine"],
    "Mango": ["Guava", "Litchi", "Banana"],
    "Rose": ["Marigold", "Jasmine", "Tulsi"],
    "Tomato": ["Potato", "Cauliflower", "Cucumber"],
    "Pea": ["Wheat", "Mustard", "Gram"],
    "Marigold": ["Rose", "Tulsi", "Jasmine"],
    "Banana": ["Mango", "Guava", "Litchi"],
    "Mint": ["Tulsi", "Jasmine", "Marigold"],
    "Apple": ["Guava", "Litchi", "Mango"],
    "Onion": ["Potato", "Cauliflower", "Tomato"],
    "Neem": ["Peepal", "Mango", "Guava"],
    "Lotus": ["Rice", "Jasmine", "Marigold"],
    "Guava": ["Mango", "Litchi", "Banana"],
    "Maize": ["Wheat", "Gram", "Mustard"],
    "Gram": ["Rice", "Moong", "Sugarcane"],
    "Mustard": ["Rice", "Pea", "Wheat"],
    "Sugarcane": ["Wheat", "Gram", "Moong"],
    "Cucumber": ["Tomato", "Cauliflower", "Onion"],
    "Jowar": ["Bajra", "Ragi", "Maize"],
    "Bajra": ["Jowar", "Ragi", "Maize"],
    "Ragi": ["Jowar", "Bajra", "Maize"],
    "Moong": ["Wheat", "Rice", "Sugarcane"],
    "Arhar": ["Rice", "Wheat", "Moong"],
    "Potato": ["Onion", "Cauliflower", "Tomato"],
    "Cauliflower": ["Tomato", "Onion", "Potato"],
    "Jasmine": ["Rose", "Tulsi", "Marigold"],
    "Litchi": ["Mango", "Guava", "Banana"],
    "Peepal": ["Neem", "Mango", "Guava"]
}

# Crop information with nutrient, water, and economic data
crop_info = {
    "Rice": {"category": "Crop", "info": "Staple food, rich in carbohydrates. Requires 150-200 cm rainfall, 24°C, clayey loamy soil. Kharif crop. Used in biryani, pulao. Uttar Pradesh is India's top rice producer, especially in eastern regions like Varanasi.",
             "nutrients": "N: 80-120 kg/ha, P: 40-60 kg/ha, K: 40-60 kg/ha", "water": "1200-1500 mm", "yield": "4-6 t/ha", "price": "₹20-25/kg"},
    "Wheat": {"category": "Crop", "info": "Used for bread, chapati. Needs 20-100 cm rainfall, 17-20°C, loamy soil. Rabi crop. Uttar Pradesh leads wheat production, especially in western regions like Meerut.",
              "nutrients": "N: 100-150 kg/ha, P: 50-70 kg/ha, K: 40-60 kg/ha", "water": "400-500 mm", "yield": "3-5 t/ha", "price": "₹22-28/kg"},
    "Tulsi": {"category": "Herb", "info": "Medicinal holy plant. Grows in 25-60 cm rainfall, 20-27°C, loamy soil. Used in tea, ayurveda. Kharif/summer. Widely revered in Varanasi.",
              "nutrients": "N: 20-40 kg/ha, P: 10-20 kg/ha, K: 10-20 kg/ha", "water": "200-300 mm", "yield": "0.5-1 t/ha", "price": "₹100-150/kg"},
    "Mango": {"category": "Fruit", "info": "King of fruits, varieties like Dasheri, Langra. Needs 50-100 cm rainfall, 21-26°C, well-drained soil. Summer crop. Major in Lucknow, Malihabad.",
              "nutrients": "N: 50-80 kg/ha, P: 20-30 kg/ha, K: 40-60 kg/ha", "water": "500-800 mm", "yield": "5-10 t/ha", "price": "₹50-80/kg"},
    "Rose": {"category": "Flower", "info": "Used for decoration, perfume. Grows in 25-60 cm rainfall, 20-27°C, loamy soil. All seasons. Popular in Kanpur for garlands.",
             "nutrients": "N: 30-50 kg/ha, P: 20-30 kg/ha, K: 20-30 kg/ha", "water": "300-400 mm", "yield": "2-3 t/ha", "price": "₹200-300/kg"},
    "Tomato": {"category": "Vegetable", "info": "Rich in Vitamin C. Needs 75-100 cm rainfall, 21-26°C, loamy soil. Kharif/zaid. Common in Agra markets.",
               "nutrients": "N: 100-120 kg/ha, P: 50-60 kg/ha, K: 60-80 kg/ha", "water": "600-800 mm", "yield": "20-30 t/ha", "price": "₹15-25/kg"},
    "Pea": {"category": "Pulse", "info": "Protein-rich pulse. Needs 25-60 cm rainfall, 20-27°C, sandy-loamy soil. Rabi crop. Used in sabzi, snacks in Gorakhpur.",
            "nutrients": "N: 20-40 kg/ha, P: 40-60 kg/ha, K: 20-30 kg/ha", "water": "300-400 mm", "yield": "2-3 t/ha", "price": "₹30-40/kg"},
    "Marigold": {"category": "Flower", "info": "Used in garlands, worship. Grows in 25-60 cm rainfall, 20-27°C, loamy soil. Kharif/summer. Essential for festivals in Ayodhya.",
                 "nutrients": "N: 30-50 kg/ha, P: 20-30 kg/ha, K: 20-30 kg/ha", "water": "300-400 mm", "yield": "2-3 t/ha", "price": "₹150-200/kg"},
    "Banana": {"category": "Fruit", "info": "Energy-rich fruit. Needs 75-150 cm rainfall, 20-30°C, loamy soil. Kharif crop. Leaves used in rituals, grown in Bareilly.",
               "nutrients": "N: 150-200 kg/ha, P: 50-70 kg/ha, K: 200-250 kg/ha", "water": "1200-1500 mm", "yield": "15-25 t/ha", "price": "₹10-15/kg"},
    "Mint": {"category": "Herb", "info": "Refreshing leaves for chutney, tea. Grows in 25-60 cm rainfall, 20-27°C, loamy soil. All seasons. Common in Lucknow cuisine.",
             "nutrients": "N: 20-40 kg/ha, P: 10-20 kg/ha, K: 10-20 kg/ha", "water": "200-300 mm", "yield": "0.5-1 t/ha", "price": "₹80-120/kg"},
    "Apple": {"category": "Fruit", "info": "Nutritious fruit. Needs 50-100 cm rainfall, 15-20°C, loamy soil. Rabi/summer. Limited to cooler hilly areas of Uttar Pradesh.",
              "nutrients": "N: 50-80 kg/ha, P: 20-30 kg/ha, K: 40-60 kg/ha", "water": "500-800 mm", "yield": "5-10 t/ha", "price": "₹80-120/kg"},
    "Onion": {"category": "Vegetable", "info": "Essential for cooking. Needs 25-60 cm rainfall, 20-27°C, loamy soil. Rabi/kharif. Widely grown in Aligarh.",
              "nutrients": "N: 80-100 kg/ha, P: 40-50 kg/ha, K: 50-60 kg/ha", "water": "400-500 mm", "yield": "15-20 t/ha", "price": "₹20-30/kg"},
    "Neem": {"category": "Tree", "info": "Medicinal tree, repels insects. Grows in 50-100 cm rainfall, 20-30°C, any soil. All seasons. Common in rural Uttar Pradesh.",
             "nutrients": "N: 10-20 kg/ha, P: 10-15 kg/ha, K: 10-15 kg/ha", "water": "200-300 mm", "yield": "1-2 t/ha", "price": "₹50-100/kg"},
    "Lotus": {"category": "Flower", "info": "National flower, grows in water. Needs 75-150 cm rainfall, 20-30°C, muddy soil. Kharif/summer. Sacred in Prayagraj.",
              "nutrients": "N: 20-30 kg/ha, P: 10-20 kg/ha, K: 10-20 kg/ha", "water": "1000-1500 mm", "yield": "0.5-1 t/ha", "price": "₹200-300/kg"},
    "Guava": {"category": "Fruit", "info": "Vitamin C-rich fruit. Needs 50-100 cm rainfall, 20-30°C, loamy soil. Kharif crop. Popular in Allahabad (Prayagraj).",
              "nutrients": "N: 50-80 kg/ha, P: 20-30 kg/ha, K: 40-60 kg/ha", "water": "500-800 mm", "yield": "10-15 t/ha", "price": "₹30-50/kg"},
    "Maize": {"category": "Crop", "info": "Versatile cereal for food, fodder. Needs 50-100 cm rainfall, 21-27°C, loamy soil. Kharif crop. Grown in western Uttar Pradesh.",
              "nutrients": "N: 100-150 kg/ha, P: 50-70 kg/ha, K: 40-60 kg/ha", "water": "500-800 mm", "yield": "4-6 t/ha", "price": "₹15-20/kg"},
    "Gram": {"category": "Pulse", "info": "Protein-rich pulse (chana). Needs 25-60 cm rainfall, 20-27°C, sandy-loamy soil. Rabi crop. Common in Jhansi.",
             "nutrients": "N: 20-40 kg/ha, P: 40-60 kg/ha, K: 20-30 kg/ha", "water": "300-400 mm", "yield": "1-2 t/ha", "price": "₹50-60/kg"},
    "Mustard": {"category": "Oilseed", "info": "Used for oil, spice. Needs 25-60 cm rainfall, 15-25°C, loamy soil. Rabi crop. Major in Saharanpur.",
                "nutrients": "N: 60-80 kg/ha, P: 30-40 kg/ha, K: 20-30 kg/ha", "water": "300-400 mm", "yield": "1-2 t/ha", "price": "₹50-70/kg"},
    "Sugarcane": {"category": "Crop", "info": "Primary sugar source. Needs 75-150 cm rainfall, 21-27°C, loamy soil. Kharif crop. Uttar Pradesh is India's top producer, especially in Muzaffarnagar.",
                  "nutrients": "N: 150-200 kg/ha, P: 60-80 kg/ha, K: 80-100 kg/ha", "water": "1500-2000 mm", "yield": "60-80 t/ha", "price": "₹3-5/kg"},
    "Cucumber": {"category": "Vegetable", "info": "Refreshing vegetable. Needs 25-60 cm rainfall, 20-30°C, loamy soil. Zaid/kharif. Common in Moradabad.",
                 "nutrients": "N: 80-100 kg/ha, P: 40-50 kg/ha, K: 50-60 kg/ha", "water": "400-500 mm", "yield": "10-15 t/ha", "price": "₹15-25/kg"},
    "Jowar": {"category": "Millet", "info": "Nutritious millet for roti. Needs 25-60 cm rainfall, 20-30°C, loamy soil. Kharif crop. Grown in Bundelkhand region like Jhansi.",
              "nutrients": "N: 60-80 kg/ha, P: 30-40 kg/ha, K: 20-30 kg/ha", "water": "300-400 mm", "yield": "1-2 t/ha", "price": "₹25-35/kg"},
    "Bajra": {"category": "Millet", "info": "Drought-resistant millet. Needs 25-60 cm rainfall, 20-30°C, sandy soil. Kharif crop. Suitable for arid areas like Agra.",
              "nutrients": "N: 60-80 kg/ha, P: 30-40 kg/ha, K: 20-30 kg/ha", "water": "200-300 mm", "yield": "1-2 t/ha", "price": "₹20-30/kg"},
    "Ragi": {"category": "Millet", "info": "Rich in calcium, used for porridge. Needs 25-60 cm rainfall, 20-30°C, loamy soil. Kharif crop. Emerging in Uttar Pradesh.",
             "nutrients": "N: 60-80 kg/ha, P: 30-40 kg/ha, K: 20-30 kg/ha", "water": "300-400 mm", "yield": "1-2 t/ha", "price": "₹30-40/kg"},
    "Moong": {"category": "Pulse", "info": "Protein-rich dal. Needs 25-60 cm rainfall, 20-27°C, sandy-loamy soil. Kharif/rabi. Popular in eastern Uttar Pradesh like Azamgarh.",
              "nutrients": "N: 20-40 kg/ha, P: 40-60 kg/ha, K: 20-30 kg/ha", "water": "300-400 mm", "yield": "1-1.5 t/ha", "price": "₹60-80/kg"},
    "Arhar": {"category": "Pulse", "info": "Used for dal. Needs 25-60 cm rainfall, 20-27°C, sandy-loamy soil. Kharif crop. Common in Azamgarh.",
              "nutrients": "N: 20-40 kg/ha, P: 40-60 kg/ha, K: 20-30 kg/ha", "water": "300-400 mm", "yield": "1-1.5 t/ha", "price": "₹70-90/kg"},
    "Potato": {"category": "Vegetable", "info": "Starchy tuber. Needs 25-60 cm rainfall, 15-20°C, loamy soil. Rabi crop. Major in Farrukhabad, Kannauj.",
               "nutrients": "N: 100-120 kg/ha, P: 50-60 kg/ha, K: 60-80 kg/ha", "water": "400-500 mm", "yield": "20-30 t/ha", "price": "₹10-20/kg"},
    "Cauliflower": {"category": "Vegetable", "info": "Rich in vitamins. Needs 25-60 cm rainfall, 15-20°C, loamy soil. Rabi crop. Grown in Meerut, Bulandshahr.",
                    "nutrients": "N: 80-100 kg/ha, P: 40-50 kg/ha, K: 50-60 kg/ha", "water": "400-500 mm", "yield": "15-20 t/ha", "price": "₹20-30/kg"},
    "Jasmine": {"category": "Flower", "info": "Fragrant flower for perfumes, garlands. Grows in 25-60 cm rainfall, 20-27°C, loamy soil. All seasons. Popular in Mathura.",
                "nutrients": "N: 30-50 kg/ha, P: 20-30 kg/ha, K: 20-30 kg/ha", "water": "300-400 mm", "yield": "1-2 t/ha", "price": "₹200-300/kg"},
    "Litchi": {"category": "Fruit", "info": "Sweet juicy fruit. Needs 50-100 cm rainfall, 20-30°C, loamy soil. Summer crop. Grown in Deoria, Muzaffarnagar.",
               "nutrients": "N: 50-80 kg/ha, P: 20-30 kg/ha, K: 40-60 kg/ha", "water": "500-800 mm", "yield": "5-10 t/ha", "price": "₹60-100/kg"},
    "Peepal": {"category": "Tree", "info": "Sacred tree, provides shade. Grows in 50-100 cm rainfall, 20-30°C, any soil. All seasons. Common in Varanasi, Ayodhya.",
               "nutrients": "N: 10-20 kg/ha, P: 10-15 kg/ha, K: 10-15 kg/ha", "water": "200-300 mm", "yield": "1-2 t/ha", "price": "₹50-100/kg"}
}

# City-specific average conditions
city_conditions = {
    "Lucknow": {"temp": 25, "humidity": 75, "rainfall": 200}, "Kanpur": {"temp": 28, "humidity": 70, "rainfall": 180},
    "Varanasi": {"temp": 26, "humidity": 80, "rainfall": 220}, "Agra": {"temp": 30, "humidity": 60, "rainfall": 150},
    "Meerut": {"temp": 27, "humidity": 65, "rainfall": 170}, "Prayagraj": {"temp": 24, "humidity": 78, "rainfall": 190},
    "Ghaziabad": {"temp": 26, "humidity": 70, "rainfall": 160}, "Jhansi": {"temp": 29, "humidity": 62, "rainfall": 140},
    "Aligarh": {"temp": 27, "humidity": 68, "rainfall": 175}, "Bareilly": {"temp": 25, "humidity": 72, "rainfall": 185},
    "Gorakhpur": {"temp": 24, "humidity": 79, "rainfall": 210}, "Noida": {"temp": 27, "humidity": 66, "rainfall": 165},
    "Firozabad": {"temp": 28, "humidity": 64, "rainfall": 155}, "Mathura": {"temp": 29, "humidity": 63, "rainfall": 145},
    "Moradabad": {"temp": 26, "humidity": 71, "rainfall": 180}, "Saharanpur": {"temp": 25, "humidity": 74, "rainfall": 190},
    "Faizabad": {"temp": 24, "humidity": 77, "rainfall": 200}, "Muzaffarnagar": {"temp": 26, "humidity": 69, "rainfall": 175},
    "Rampur": {"temp": 27, "humidity": 67, "rainfall": 170}, "Azamgarh": {"temp": 25, "humidity": 76, "rainfall": 195},
    "Basti": {"temp": 24, "humidity": 78, "rainfall": 205}, "Deoria": {"temp": 25, "humidity": 77, "rainfall": 200},
    "Sitapur": {"temp": 26, "humidity": 70, "rainfall": 185}, "Lakhimpur": {"temp": 25, "humidity": 73, "rainfall": 190},
    "Hathras": {"temp": 28, "humidity": 65, "rainfall": 160}, "Mirzapur": {"temp": 24, "humidity": 79, "rainfall": 210},
    "Bulandshahr": {"temp": 27, "humidity": 66, "rainfall": 170}, "Ayodhya": {"temp": 24, "humidity": 77, "rainfall": 200},
    "Gonda": {"temp": 25, "humidity": 75, "rainfall": 195}, "Barabanki": {"temp": 26, "humidity": 71, "rainfall": 180}
}

# Simulated historical yield trends (t/ha for last 5 years)
historical_yields = {
    crop: np.random.uniform(*parse_yield(info["yield"]), 5).tolist()
    for crop, info in crop_info.items()
}

# Predict function with enhanced features
def recommend(city, temp, humidity, rain, soil, season):
    logger.info(f"Processing inputs: city={city}, temp={temp}, humidity={humidity}, rain={rain}, soil={soil}, season={season}")

    # Input validation
    if not all([city, soil, season]):
        logger.error("Missing required inputs")
        return "❌ Error: Please select City, Soil, and Season", None, None, None, None, None, None, None, None, None, None
    if temp is not None and not (0 <= temp <= 50):
        logger.error("Invalid temperature")
        return "❌ Error: Temperature must be 0-50°C", None, None, None, None, None, None, None, None, None, None
    if humidity is not None and not (0 <= humidity <= 100):
        logger.error("Invalid humidity")
        return "❌ Error: Humidity must be 0-100%", None, None, None, None, None, None, None, None, None, None
    if rain is not None and not (rain >= 0):
        logger.error("Invalid rainfall")
        return "❌ Error: Rainfall must be >=0 mm", None, None, None, None, None, None, None, None, None, None

    # Use city-specific defaults if not provided
    city_data = city_conditions.get(city, {"temp": 25, "humidity": 70, "rainfall": 180})
    temp = temp if temp is not None else city_data["temp"]
    humidity = humidity if humidity is not None else city_data["humidity"]
    rain = rain if rain is not None else city_data["rainfall"]
    logger.info(f"Using values: temp={temp}, humidity={humidity}, rain={rain}")

    # Create input dictionary
    input_dict = {"temperature": temp, "humidity": humidity, "rainfall": rain}
    for col in training_columns:
        input_dict[col] = 0
    if f"city_{city}" in training_columns:
        input_dict[f"city_{city}"] = 1
    if f"soil_{soil}" in training_columns:
        input_dict[f"soil_{soil}"] = 1
    if f"season_{season}" in training_columns:
        input_dict[f"season_{season}"] = 1

    input_df = pd.DataFrame([input_dict])[training_columns]

    # Predict probabilities
    try:
        probas = model.predict_proba(input_df)[0]
        top_indices = np.argsort(probas)[::-1][:5]
        top_crops = model.classes_[top_indices]
        top_scores = probas[top_indices]
        logger.info(f"Top crops: {top_crops.tolist()}")
    except Exception as e:
        logger.error(f"Prediction error: {e}")
        return f"❌ Error in prediction: {e}", None, None, None, None, None, None, None, None, None, None

    # Detailed recommendations with new features
    result = f"### Crop Recommendations for {city}, Soil: {soil}, Season: {season}\n\n"
    recommendations = []
    for crop, score in zip(top_crops, top_scores):
        c = crop_info.get(crop, {"category": "Unknown", "info": "No info available", "nutrients": "N/A", "water": "N/A", "yield": "N/A", "price": "N/A"})
        rotation = crop_rotation.get(crop, ["Unknown"])
        water_deficit = parse_water(c["water"]) - rain
        water_deficit = max(0, water_deficit)
        result += (
            f"✅ **{crop}**\n"
            f"📂 Category: {c['category']}\n"
            f"📘 Info: {c['info']}\n"
            f"🔢 Confidence: {round(score * 100, 2)}%\n"
            f"🌱 Nutrients: {c['nutrients']}\n"
            f"💧 Water Needed: {c['water']} (Irrigation: ~{round(water_deficit, 1)} mm)\n"
            f"📈 Yield: {c['yield']}\n"
            f"💰 Price: {c['price']}\n"
            f"🔄 Rotation: Next season try {', '.join(rotation)}\n\n"
        )
        recommendations.append({
            "Crop": crop,
            "Category": c["category"],
            "Info": c["info"],
            "Confidence": round(score * 100, 2),
            "Nutrients": c["nutrients"],
            "Water": c["water"],
            "Irrigation": round(water_deficit, 1),
            "Yield": c["yield"],
            "Price": c["price"],
            "Rotation": ", ".join(rotation)
        })

    # Bar graph
    bar_path = None
    try:
        plt.figure(figsize=(8, 5))
        sns.barplot(x=top_scores, y=top_crops, palette="viridis")
        plt.title(f"Top 5 Crop Predictions for {city}")
        plt.xlabel("Confidence")
        plt.ylabel("Crop")
        bar_path = os.path.join(output_dir, "bar.png")
        print(f"Saving bar graph to {bar_path}...")
        plt.savefig(bar_path, bbox_inches='tight', dpi=100)
        plt.close()
        if os.path.exists(bar_path):
            logger.info(f"Bar graph saved: {bar_path}")
            display(Image(filename=bar_path))
        else:
            logger.error(f"Bar graph not found at: {bar_path}")
    except Exception as e:
        logger.error(f"Error generating bar graph: {e}")

    # Pie chart
    pie_path = None
    try:
        categories = [crop_info[crop]["category"] for crop in top_crops]
        category_counts = pd.Series(categories).value_counts()
        plt.figure(figsize=(6, 6))
        plt.pie(category_counts, labels=category_counts.index, autopct='%1.1f%%', colors=sns.color_palette("pastel"))
        plt.title("Category Distribution of Top Predictions")
        pie_path = os.path.join(output_dir, "pie.png")
        print(f"Saving pie chart to {pie_path}...")
        plt.savefig(pie_path, bbox_inches='tight', dpi=100)
        plt.close()
        if os.path.exists(pie_path):
            logger.info(f"Pie chart saved: {pie_path}")
            display(Image(filename=pie_path))
        else:
            logger.error(f"Pie chart not found at: {pie_path}")
    except Exception as e:
        logger.error(f"Error generating pie chart: {e}")

    # Scatter plot
    scatter_path = None
    try:
        plt.figure(figsize=(8, 5))
        for crop in top_crops:
            crop_data = df[df["crop"] == crop]
            plt.scatter(crop_data["temperature"], crop_data["rainfall"], label=crop, s=100)
        plt.scatter([temp], [rain], c='red', marker='*', s=200, label='Input')
        plt.xlabel("Temperature (°C)")
        plt.ylabel("Rainfall (mm)")
        plt.title("Temperature vs Rainfall for Top Crops")
        plt.legend()
        scatter_path = os.path.join(output_dir, "scatter.png")
        print(f"Saving scatter plot to {scatter_path}...")
        plt.savefig(scatter_path, bbox_inches='tight', dpi=100)
        plt.close()
        if os.path.exists(scatter_path):
            logger.info(f"Scatter plot saved: {scatter_path}")
            display(Image(filename=scatter_path))
        else:
            logger.error(f"Scatter plot not found at: {scatter_path}")
    except Exception as e:
        logger.error(f"Error generating scatter plot: {e}")

    # Heatmap for feature importance
    heatmap_path = None
    try:
        feature_importance = pd.Series(model.feature_importances_, index=training_columns).sort_values(ascending=False)[:10]
        plt.figure(figsize=(8, 5))
        sns.heatmap(feature_importance.to_frame(), annot=True, cmap="YlGnBu", cbar=True)
        plt.title("Top 10 Feature Importances")
        heatmap_path = os.path.join(output_dir, "heatmap.png")
        print(f"Saving heatmap to {heatmap_path}...")
        plt.savefig(heatmap_path, bbox_inches='tight', dpi=100)
        plt.close()
        if os.path.exists(heatmap_path):
            logger.info(f"Heatmap saved: {heatmap_path}")
            display(Image(filename=heatmap_path))
        else:
            logger.error(f"Heatmap not found at: {heatmap_path}")
    except Exception as e:
        logger.error(f"Error generating heatmap: {e}")

    # Historical yield trend plot
    yield_trend_path = None
    try:
        plt.figure(figsize=(8, 5))
        for crop in top_crops:
            plt.plot(range(2020, 2025), historical_yields[crop], label=crop, marker='o')
        plt.xlabel("Year")
        plt.ylabel("Yield (t/ha)")
        plt.title("Historical Yield Trends for Top Crops")
        plt.legend()
        yield_trend_path = os.path.join(output_dir, "yield_trend.png")
        print(f"Saving yield trend plot to {yield_trend_path}...")
        plt.savefig(yield_trend_path, bbox_inches='tight', dpi=100)
        plt.close()
        if os.path.exists(yield_trend_path):
            logger.info(f"Yield trend plot saved: {yield_trend_path}")
            display(Image(filename=yield_trend_path))
        else:
            logger.error(f"Yield trend plot not found at: {yield_trend_path}")
    except Exception as e:
        logger.error(f"Error generating yield trend plot: {e}")

    # Map-based crop regions
    map_path = None
    try:
        m = folium.Map(location=[26.8467, 80.9462], zoom_start=7)  # Center of Uttar Pradesh
        for crop in top_crops[:3]:
            folium.Marker(
                location=[26.8467 + np.random.uniform(-0.5, 0.5), 80.9462 + np.random.uniform(-0.5, 0.5)],
                popup=f"{crop}: {round(probas[model.classes_ == crop][0] * 100, 2)}%",
                icon=folium.Icon(color="green" if crop_info[crop]["category"] in ["Crop", "Millet", "Pulse", "Oilseed"] else "red")
            ).add_to(m)
        map_path = os.path.join(output_dir, "map.html")
        print(f"Saving map to {map_path}...")
        m.save(map_path)
        if os.path.exists(map_path):
            logger.info(f"Map saved: {map_path}")
        else:
            logger.error(f"Map not found at: {map_path}")
    except Exception as e:
        logger.error(f"Error generating map: {e}")

    # Speech output
    speech_path = None
    try:
        speech_text = f"Crop recommendations for {city}, soil {soil}, season {season}. "
        for crop, score in zip(top_crops, top_scores):
            speech_text += f"{crop} with {round(score * 100, 2)} percent confidence. "
        tts = gTTS(text=speech_text, lang='en')
        speech_path = os.path.join(output_dir, "recommendation.mp3")
        print(f"Saving speech to {speech_path}...")
        tts.save(speech_path)
        if os.path.exists(speech_path):
            logger.info(f"Speech saved: {speech_path}")
        else:
            logger.error(f"Speech not found at: {speech_path}")
    except Exception as e:
        logger.error(f"Error generating speech: {e}")

    # PDF export
    pdf_path = None
    try:
        pdf_path = os.path.join(output_dir, "recommendations.pdf")
        c = canvas.Canvas(pdf_path, pagesize=letter)
        c.drawString(50, 750, f"Crop Recommendations for {city}, Soil: {soil}, Season: {season}")
        y = 700
        for rec in recommendations:
            c.drawString(50, y, f"Crop: {rec['Crop']}, Category: {rec['Category']}, Confidence: {rec['Confidence']}%")
            y -= 20
            c.drawString(50, y, f"Nutrients: {rec['Nutrients']}")
            y -= 20
            c.drawString(50, y, f"Water: {rec['Water']} (Irrigation: ~{rec['Irrigation']} mm)")
            y -= 20
            c.drawString(50, y, f"Yield: {rec['Yield']}, Price: {rec['Price']}")
            y -= 20
            c.drawString(50, y, f"Rotation: {rec['Rotation']}")
            y -= 20
            text = rec['Info']
            lines = []
            while len(text) > 80:
                lines.append(text[:80])
                text = text[80:]
            lines.append(text)
            for line in lines:
                c.drawString(50, y, line)
                y -= 20
            y -= 20
            if y < 50:
                c.showPage()
                y = 750
        c.save()
        print(f"Saving PDF to {pdf_path}...")
        if os.path.exists(pdf_path):
            logger.info(f"PDF saved: {pdf_path}")
        else:
            logger.error(f"PDF not found at: {pdf_path}")
    except Exception as e:
        logger.error(f"Error generating PDF: {e}")

    # Excel export
    excel_path = None
    try:
        excel_path = os.path.join(output_dir, "recommendations.xlsx")
        pd.DataFrame(recommendations).to_excel(excel_path, index=False)
        print(f"Saving Excel to {excel_path}...")
        if os.path.exists(excel_path):
            logger.info(f"Excel saved: {excel_path}")
        else:
            logger.error(f"Excel not found at: {excel_path}")
    except Exception as e:
        logger.error(f"Error generating Excel: {e}")

    # CSV export
    csv_path = None
    try:
        csv_path = os.path.join(output_dir, "recommendations.csv")
        pd.DataFrame(recommendations).to_csv(csv_path, index=False)
        print(f"Saving CSV to {csv_path}...")
        if os.path.exists(csv_path):
            logger.info(f"CSV saved: {csv_path}")
        else:
            logger.error(f"CSV not found at: {csv_path}")
    except Exception as e:
        logger.error(f"Error generating CSV: {e}")

    # Collect downloadable files
    files_to_download = [f for f in [bar_path, pie_path, scatter_path, heatmap_path, yield_trend_path, pdf_path, excel_path, csv_path, speech_path, map_path] if f and os.path.exists(f)]

    # Trigger automatic downloads
    for file in files_to_download:
        try:
            files.download(file)
            logger.info(f"Triggered download for: {file}")
        except Exception as e:
            logger.error(f"Error downloading {file}: {e}")

    # List generated files
    file_list = list_output_files()

    # Fallback manual download instructions
    manual_download = "\n\n**Manual Download Instructions** (if downloads fail):\n"
    for file in files_to_download:
        manual_download += f"Run: `files.download('{file}')`\n"
    result += manual_download

    return (result, bar_path, pie_path, scatter_path, heatmap_path, yield_trend_path, map_path, speech_path, pdf_path, excel_path, csv_path, file_list)

# Gradio interface
try:
    iface = gr.Interface(
        fn=recommend,
        inputs=[
            gr.Dropdown(list(city_conditions.keys()), label="🏙️ City in Uttar Pradesh", value="Lucknow"),
            gr.Number(label="🌡️ Temperature (°C)", value=None, info="Leave blank for city default (0-50°C)"),
            gr.Number(label="💧 Humidity (%)", value=None, info="Leave blank for city default (0-100%)"),
            gr.Number(label="🌧️ Rainfall (mm)", value=None, info="Leave blank for city default (≥0 mm)"),
            gr.Dropdown(['clay', 'sandy', 'loamy', 'black', 'red'], label="🧱 Soil Type", value="clay"),
            gr.Dropdown(['kharif', 'rabi', 'summer'], label="📅 Season", value="kharif")
        ],
        outputs=[
            gr.Markdown(label="📋 Recommended Crops"),
            gr.Image(label="📊 Bar Graph", type="filepath"),
            gr.Image(label="🥧 Pie Chart", type="filepath"),
            gr.Image(label="📈 Scatter Plot", type="filepath"),
            gr.Image(label="🔥 Feature Importance Heatmap", type="filepath"),
            gr.Image(label="📈 Historical Yield Trend", type="filepath"),
            gr.HTML(label="🗺️ Crop Region Map"),
            gr.Audio(label="🎙️ Speech Output", type="filepath"),
            gr.File(label="📥 Download PDF Report"),
            gr.File(label="📥 Download Excel Report"),
            gr.File(label="📥 Download CSV Report"),
            gr.Textbox(label="📂 Generated Files")
        ],
        title="🌾 Enhanced Crop Recommender for Uttar Pradesh",
        description="Input city, environmental factors, soil, and season to get crop recommendations with nutrient requirements, water needs, yield, price, rotation suggestions, historical trends, and downloadable reports (PDF, Excel, CSV) for Uttar Pradesh."
    )
except Exception as e:
    logger.error(f"Error creating Gradio interface: {e}")
    raise

# Launch in Colab with Gradio sharing or ngrok
try:
    logger.info("Attempting to launch Gradio with share=True")
    public_url = iface.launch(share=True, prevent_thread_lock=True)
    logger.info(f"Gradio interface running at: {public_url}")
except Exception as e:
    logger.warning(f"Gradio share=True failed: {e}. Falling back to ngrok.")
    try:
        kill_process_on_port(7860)
        port = find_available_port(7860, 7870)
        if port is None:
            logger.error("No available ports in range 7860-7870")
            raise OSError("No available ports in range 7860-7870. Try restarting the runtime.")
        ngrok_api_key = "YOUR_NGROK_API_KEY"  # Replace with your ngrok API key
        terminate_ngrok_sessions(ngrok_api_key)
        ngrok.set_auth_token("305xql2LT9jVOeTHjRTmSFJ3yAP_4ZyFmpyY9YBRV8vCko8y3")
        public_url = ngrok.connect(port)
        logger.info(f"Gradio interface running at: {public_url}")
        iface.launch(share=False, server_port=port)
    except Exception as e:
        logger.error(f"Error launching Gradio with ngrok: {e}. Try restarting the runtime or using a different port.")
        print("Gradio launch failed. Try the following steps:")
        print("1. Restart the runtime: Runtime > Restart runtime")
        print("2. Terminate ngrok sessions at https://dashboard.ngrok.com/agents")
        print("3. Verify ngrok authtoken at https://dashboard.ngrok.com/get-started/your-authtoken")
        print("4. Reinstall ngrok: !pip install pyngrok")
        print("5. Relaunch with a different port: iface.launch(share=False, server_port=7870)")
        raise

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.2.1
    Uninstalling click-8.2.1:
      Successfully uninstalled click-8.2.1
